In [60]:
import os
current_directory = str(os.getcwd())

In [61]:
import numpy as np
from sympy import *
from fractions import Fraction
import pickle
from scipy.interpolate import griddata
import pandas as pd


#converting from 2nd unit to 1st
pc_kpc = 1e3  # number of pc in one kpc
cm_km = 1e5  # number of cm in one km
s_day = 24*3600  # number of seconds in one day
s_min = 60  # number of seconds in one hour
s_hr = 3600  # number of seconds in one hour
cm_Rsun = 6.957e10  # solar radius in cm
g_Msun = 1.989e33  # solar mass in g
cgs_G = 6.674e-8
cms_c = 2.998e10
g_mH = 1.6736e-24
g_me = 9.10938e-28
cgs_h = 6.626e-27
deg_rad = 180e0/np.pi
arcmin_deg = 60e0
arcsec_deg = 3600e0
cm_kpc = 3.086e+21  # number of centimeters in one parsec
cm_pc = cm_kpc/1e+3
s_Myr = 1e+6*(365*24*60*60)  # megayears to seconds

#REQUIRED FUNCTIONS
###########################################################################################################################################
#importing data from csv file to np array
def file_reader(list_of_file_names):
    dataframe_list=[]
    for i in list_of_file_names:
        df = pd.read_csv(current_directory+r'\{}.dat'.format(i),delimiter=',')
        dataframe_list.append(df)
    return dataframe_list

#extrapolation
def interpolation(list1,list2,standard):
    interpolated_data = griddata(list1, list2, standard, method='linear', fill_value=nan, rescale=False)
    return interpolated_data
###########################################################################################################################################
os.chdir(current_directory+'\M31_data')
raw_data = pd.read_csv('combined_data_m31.csv', skiprows=1)
os.chdir(current_directory)

In [62]:

def find_and_multiply_column(dataframe, substring, multiplier):
    # Create a copy of the DataFrame to avoid modifying the original
    result_df = dataframe.copy()
    i = 0
    for col in result_df.columns:
        if substring in col:
            try:
                result_df[col] = result_df[col] * multiplier[i]
            except:
                result_df[col] = result_df[col] * multiplier
            i+=1
    return result_df

def keep_substring_columns(dataframe, substring):
    # Get the columns that contain the specified substring
    filtered_columns = [col for col in dataframe.columns if substring in col]
    
    # Create a new DataFrame with only the filtered columns
    result_df = dataframe[filtered_columns].copy()
    
    return result_df, filtered_columns

radii_df = keep_substring_columns(raw_data, 'r ')[0]
#radii_df = radii_df.drop(columns='error kms')

#to obtain radius data from every df
distance_m31= 0.78 #Mpc
distances_Mpc=np.array([0.785,0.785,0.780,0.780])

#convert arcmin to kpc
radii_df = find_and_multiply_column(radii_df, 'r arcmin', ((distance_m31*1000)/(arcmin_deg*deg_rad)))
#convert arcsec to kpc
radii_df = find_and_multiply_column(radii_df, 'r arcsec', ((distance_m31*1000)/(arcsec_deg*deg_rad)))
#distance correction
radii_df = find_and_multiply_column(radii_df, 'r kpc', distance_m31/distances_Mpc)

# print("Original DataFrame:")
#print(raw_data)
print("\nDataFrame after multiplication:")
radii_df


DataFrame after multiplication:


,r kpc,r arcmin,r kpc.1,r arcmin.1,r kpc.2,r kpc.3,r arcmin.2
0,0.000000,0.378911,0.000000,0.378911,0.00,6.25,1.295558
1,0.476943,0.755553,0.476943,0.755553,0.48,6.75,1.530165
2,0.953885,1.134464,0.953885,1.134464,0.96,7.25,1.749570
3,1.430828,1.513375,1.430828,1.513375,1.44,7.75,1.973514
4,1.907771,1.890017,1.907771,1.890017,1.92,8.25,2.198137
...,...,...,...,...,...,...,...
95,45.309554,36.302848,45.309554,36.302848,45.60,NaN,NaN
96,NaN,36.681759,NaN,36.681759,NaN,NaN,NaN
97,NaN,37.058401,NaN,37.058401,NaN,NaN,NaN
98,NaN,37.437312,NaN,37.437312,NaN,NaN,NaN


In [63]:
raw_data[radii_df.columns] = radii_df

In [64]:
# get the next or previous column of the dataframe
def get_adjacent_column(df, col, next = True):
    index_of_target = df.columns.get_loc(col)
    if index_of_target < len(df.columns) - 1:
        if next:
            return df.columns[index_of_target+1]
        else:
            return df.columns[index_of_target-1]
    else:
        print("The target column is the last column.")

# Find the column with the maximum number of NaN values
coarsest_radii_mask = radii_df.isnull().sum().idxmax()

print("Coarsest radii is {} and the data it corresponds to is {}:".format(coarsest_radii_mask,get_adjacent_column(raw_data,coarsest_radii_mask)))
kpc_r = radii_df[coarsest_radii_mask].to_numpy()


Coarsest radii is r kpc.3 and the data it corresponds to is sigma_sfr:


In [65]:
obs_column_names = ["Radius(kpc)", r"\Sigma_{tot}", r"\Sigma_{gas}", "q", "\Omega", r"\Sigma_{sfr}", "T"]
# interpolation for the whole data_frame

def df_interpolation(df, radii_df, standard):
    result_df = df.copy()
    for cols in radii_df.columns:
        result_df[get_adjacent_column(df, cols)] = interpolation(df[cols],df[get_adjacent_column(df, cols)],standard)
        result_df.drop(columns=[cols], inplace=True)
    result_df.insert(0, 'kpc_r', standard)
    return result_df

interpolated_df = df_interpolation(raw_data,radii_df, kpc_r)
nan_mask = np.isnan(interpolated_df)

In [66]:
interpolated_df = interpolated_df[~(nan_mask.sum(axis=1)>0)]
#interpolated_df.dropna()


In [67]:
def inclination_correction(df, i_new, i_old):
    df = df.copy()
    df= df.iloc[:,1:]*np.cos(i_new)/np.cos(i_old)
    return df
i_m31= 75 #deg
inclinations=np.array([i_m31,77,i_m31,77,i_m31,75,77.5]) #used i_m31 as no inclination correction is needed for Claude data
inclination_correction(interpolated_df, np.radians(i_m31), np.radians(inclinations))

,sigma_tot,sigma_HI_chemin,sigma_HI_claude,vcirc_chemin kms,vcirc_claude kms,sigma_sfr,molfrac
0,258.031197,1.866799,2.690412,270.438222,236.628958,0.40,0.354183
1,240.662380,1.641848,2.802953,275.524273,235.657500,0.43,0.229933
2,219.378259,1.933217,2.860923,272.269533,235.903333,0.52,0.165500
3,197.137487,2.820427,3.026821,279.571236,235.462500,0.41,0.194023
4,180.420913,4.214478,3.316410,285.299954,233.159375,0.43,0.165513
5,171.574052,5.268530,3.653028,289.904137,230.477917,0.56,0.188228
6,163.684522,4.728120,4.027901,294.160450,229.280625,0.68,0.206946
7,158.900000,3.952089,4.323198,297.944654,228.308750,0.75,0.250353
8,156.002791,3.661155,4.527863,302.232339,229.146667,0.86,0.248157
9,148.361191,4.354184,4.663612,306.968649,232.062708,1.00,0.251231


## Specific to galaxies

In [68]:
# M31
data_choices = ['chemin', 'claude']
data_chosen=data_choices[0] #Claude data chosen
interpolated_df.drop(columns=keep_substring_columns(interpolated_df, data_chosen)[1],inplace=True)
interpolated_df
def molfrac_to_H2(df):
    df = df.copy()
    molfrac_data = keep_substring_columns(df, 'molfrac')
    if molfrac_data[0].empty:
        return
    else:
        HI_data = keep_substring_columns(interpolated_df, 'HI')
        sigma_H2 = HI_data[0].multiply((1/(1-molfrac_data[0])).values, axis = 0)
        index_of_HI = df.columns.get_loc(HI_data[1][0])
        df.insert(index_of_HI+1, 'sigma_H2', sigma_H2)
        df.drop(columns=molfrac_data[1], inplace=True)
        return df

interpolated_df = molfrac_to_H2(interpolated_df)

def add_temp(m, c, df):
    r = df.iloc[:,0].to_numpy().flatten()
    T = m*r +c
    df.insert(len(df.columns), 'T', T)
    return

add_temp(0.017e+4,0.5e+4,interpolated_df)


In [69]:
interpolated_df

,kpc_r,sigma_tot,sigma_HI_claude,sigma_H2,vcirc_claude kms,sigma_sfr,T
0,6.25,258.031197,2.690412,3.822629,236.628958,0.40,6062.5
1,6.75,240.662380,2.802953,3.470218,235.657500,0.43,6147.5
2,7.25,219.378259,2.860923,3.320480,235.903333,0.52,6232.5
3,7.75,197.137487,3.026821,3.613050,235.462500,0.41,6317.5
4,8.25,180.420913,3.316410,3.849181,233.159375,0.43,6402.5
5,8.75,171.574052,3.653028,4.335459,230.477917,0.56,6487.5
6,9.25,163.684522,4.027901,4.870852,229.280625,0.68,6572.5
7,9.75,158.900000,4.323198,5.467971,228.308750,0.75,6657.5
8,10.25,156.002791,4.527863,5.713561,229.146667,0.86,6742.5
9,10.75,148.361191,4.663612,5.904010,232.062708,1.00,6827.5


In [70]:
def replace_conversion(df, substring_to_replace, replacement_string):
    # Create a dictionary for column renaming
    rename_dict = {col: col.replace(substring_to_replace, replacement_string) for col in df.columns}
    # Rename the columns using the dictionary
    updated_df = df.rename(columns=rename_dict)

    return updated_df

conv_factors=np.array([1, (g_Msun/(cm_pc**2) ), g_Msun/(cm_pc**2), g_Msun/(cm_pc**2), cm_km,
              g_Msun/((s_Myr*1e3)*(cm_pc**2)),1])
interpolated_df = interpolated_df*conv_factors
#interpolated_df= replace_conversion(interpolated_df, 'kpc', 'cm')
interpolated_df= replace_conversion(interpolated_df, 'kms', 'cms')


In [71]:
def vcirc_to_qomega(df):
    df = df.copy()
    vcirc_data = keep_substring_columns(df, 'vcirc')
    if vcirc_data[0].empty:
        return
    else:
        r = df.iloc[:,0].to_numpy().flatten()*cm_kpc
        Om = vcirc_data[0].to_numpy().flatten()/r
        q = -1 * r/Om* np.gradient(Om)/np.gradient(r)
        index_of_vcirc = df.columns.get_loc(vcirc_data[1][0])
        df.insert(index_of_vcirc, 'q', q)
        df.insert(index_of_vcirc, '\Omega', Om)
        df.drop(columns=vcirc_data[1], inplace=True)
        return df
interpolated_df = vcirc_to_qomega(interpolated_df)


In [72]:
interpolated_df

,kpc_r,sigma_tot,sigma_HI_claude,sigma_H2,\Omega,q,sigma_sfr,T
0,6.25,0.053891,0.000562,0.000798,1.226851e-15,0.973442,2.649088e-21,6062.5
1,6.75,0.050263,0.000585,0.000725,1.131310e-15,1.029013,2.847769e-21,6147.5
2,7.25,0.045818,0.000598,0.000693,1.054387e-15,1.009338,3.443814e-21,6232.5
3,7.75,0.041173,0.000632,0.000755,9.845191e-16,1.090901,2.715315e-21,6317.5
4,8.25,0.037682,0.000693,0.000804,9.158050e-16,1.179895,2.847769e-21,6402.5
5,8.75,0.035834,0.000763,0.000905,8.535429e-16,1.154244,3.708723e-21,6487.5
6,9.25,0.034186,0.000841,0.001017,8.032111e-16,1.091192,4.503449e-21,6572.5
7,9.75,0.033187,0.000903,0.001142,7.587907e-16,1.012347,4.967039e-21,6657.5
8,10.25,0.032582,0.000946,0.001193,7.244255e-16,0.838611,5.695538e-21,6742.5
9,10.75,0.030986,0.000974,0.001233,6.995213e-16,0.720961,6.622719e-21,6827.5
